<a href="https://colab.research.google.com/github/Sara1041/Thesis/blob/main/OneShortLearning_ResultComparison_WithTestDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install opencv-python &> /dev/null
!apt update &> /dev/null
!apt install ffmpeg libsm6 libxext6  -y &> /dev/null

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
import numpy as np
import os
from numpy import genfromtxt
import cv2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from sklearn.preprocessing import normalize
K.set_image_data_format('channels_first')
import random
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.utils import plot_model
import sys
import pandas as pd
import glob

2023-01-09 11:08:51.239483: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2023-01-09 11:08:53.843939: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 11:08:53.849310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 11:08:53.849511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [ ]:
def triplet_loss_t(y_true,y_pred):
    anchor=y_pred[:,0:128]
    pos=y_pred[:,128:256]
    neg=y_pred[:,256:384]
    positive_distance = K.sum(K.abs(anchor-pos), axis=1)
    negative_distance = K.sum(K.abs(anchor-neg), axis=1)
    probs=K.softmax([positive_distance,negative_distance],axis=0)
    loss=K.mean(K.abs(probs[0])+K.abs(1.0-probs[1]))
    return loss

In [ ]:
model=tf.keras.models.load_model('triplet_model.h5',custom_objects={'triplet_loss_t':triplet_loss_t})

2023-01-09 11:08:54.220022: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 11:08:54.220587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 11:08:54.220998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-09 11:08:54.221237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

In [ ]:
def image_resizing(image):
    image=cv2.resize(image,(96,96))

    return image

def encode_img(img1,triplet_model):
    #img1=cv2.imread(path,1)
    img=img1[...,::-1]
    img=np.around(np.transpose(img,(2,0,1))/255,decimals=12)
    x_train=np.array([img])
    emb=triplet_model.layers[3].predict_on_batch(x_train)
    return emb

threshold=.8
interval=0.3
def confidence_value(ref_encode,img_encode,thres=threshold):
    #diff=np.max(img_encode-ref_encode)
    dist=np.linalg.norm((img_encode-ref_encode))
    #confidence=(1-K.eval(tf.minimum(dist,1)))
    confidence=(threshold-max([dist,interval]))/(threshold-interval)
    return dist,confidence

In [ ]:
df = pd.read_csv('test.csv')
df.head()

,ref,new,match
0,test/541/541_script.BMP,test/541/541_CR.BMP,1
1,test/541/541_script.BMP,test/582/582_CR.BMP,0
2,test/542/542_script.BMP,test/542/542_Obl.BMP,1
3,test/542/542_script.BMP,test/584/584_Obl.BMP,0
4,test/543/543_script.BMP,test/543/543_Obl.BMP,1


In [ ]:
y_true = df['match']
y_pred = []
for index, row in df.iterrows():
    rimage=cv2.imread(row["ref"])
    image=cv2.imread(row["new"])
    rimg=image_resizing(rimage)
    img=image_resizing(image)
    r_encode=encode_img(rimg,model)
    img_encode=encode_img(img,model)
    dist,conf=confidence_value(r_encode,img_encode)
    if dist < threshold:
        y_pred.append(1)
    else:
        y_pred.append(0)

2023-01-09 11:09:00.571472: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-01-09 11:09:01.156569: I tensorflow/stream_executor/cuda/cuda_blas.cc:1633] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9608    0.8448    0.8991        58
           1     0.8657    0.9667    0.9134        60

    accuracy                         0.9068       118
   macro avg     0.9132    0.9057    0.9062       118
weighted avg     0.9124    0.9068    0.9064       118

